# 1. Connexion Spark à PostgreSQL et Chargement des Données

In [1]:
from pyspark.sql import SparkSession

# Initialisation de la session Spark avec le driver PostgreSQL
spark = SparkSession.builder \
    .appName("PostgresConnection") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.postgresql:postgresql:42.2.25") \
    .getOrCreate()

# URL JDBC pour PostgreSQL
jdbc_url = "jdbc:postgresql://192.168.1.24:5442/postgres"

# Propriétés JDBC
jdbc_properties = {
    "user": "postgres",
    "password": "test2021",
    "driver": "org.postgresql.Driver"
}

# Charger les données historiques depuis la table 'toots' (changement ici)
try:
    df_toots = spark.read.jdbc(url=jdbc_url, table="toots", properties=jdbc_properties)
    df_toots.show(5)  # Afficher les premières lignes pour vérifier
except Exception as e:
    print("Erreur rencontrée:", e)

# Charger les données historiques depuis la table 'toots' (changement ici)
try:
    df_hashtags = spark.read.jdbc(url=jdbc_url, table="hashtags", properties=jdbc_properties)
    df_hashtags.show(5)  # Afficher les premières lignes pour vérifier
except Exception as e:
    print("Erreur rencontrée:", e)

+---------------+--------------------+--------------------+--------+----------+-----+
|           user|             content|           timestamp|language|      date| time|
+---------------+--------------------+--------------------+--------+----------+-----+
|    ezosnapshot|<p>Photographers ...| 2024-10-04 07:23:20|      en|2024-10-04|07:23|
|       knotfeed|<p>⚡️At least 177...| 2024-10-04 07:38:29|      en|2024-10-04|07:38|
|            344|<p>Interest rates...| 2024-10-04 07:38:31|      en|2024-10-04|07:38|
|realTuckFrumper|<p>After &#39;RRR...|2024-10-04 07:38:...|      en|2024-10-04|07:38|
|           hkrn|<p>Experimental w...| 2024-10-04 07:38:34|      en|2024-10-04|07:38|
+---------------+--------------------+--------------------+--------+----------+-----+
only showing top 5 rows

+--------------------+--------------------+--------------------+--------+----+-----+-----------+-------+
|                user|             content|           timestamp|language|date| time|toot_length|

# 2. Filtrer les toots en fonction de l'activité des utilisateurs

In [2]:
from pyspark.sql import functions as F

# Compter les toots par utilisateur
df_user_activity = df_toots.groupBy("user").agg(F.count("*").alias("toot_count"))

# Filtrer les utilisateurs avec plus de X toots (par exemple 10)
df_active_users = df_user_activity.filter(F.col("toot_count") > 10)

# Ordonner par le nombre de toots en ordre décroissant
df_active_users = df_active_users.orderBy(F.col("toot_count").desc())

# Afficher le résultat
df_active_users.show()


+--------------------+----------+
|                user|toot_count|
+--------------------+----------+
|                 344|       940|
|       italy24_press|       284|
|                 rff|       229|
|     NewsDailyArabic|       181|
|      AboveMaidstone|       172|
|      spotifyartists|       172|
|             prtimes|       161|
|              linguo|       131|
|          rogue_corq|       121|
|       vertx_project|       120|
|            fansided|       112|
|               folha|       103|
|           the_hindu|        90|
|            walknews|        90|
|      arXiv_csLG_bot|        88|
|     realTuckFrumper|        87|
|         Nekowashi_R|        87|
|           bloomberg|        84|
|ecf.dcd.uscourts.gov|        84|
|      arXiv_csCL_bot|        84|
+--------------------+----------+
only showing top 20 rows



# 3. Regrouper par Temps et Hashtags, Calculer le Nombre de Toots, et Identifier le Hashtag le Plus Fréquent

In [5]:
# Vérifiez d'abord le schéma pour confirmer les colonnes
df_hashtags.printSchema()

# Grouper par hashtag et compter le nombre total de toots
df_grouped = df_hashtags.groupBy("hashtag").agg(
    F.count("*").alias("toot_count")
)

# Identifier le hashtag le plus fréquent
df_top_hashtag = df_grouped.orderBy(F.desc("toot_count")).limit(10)

df_top_hashtag.show()

root
 |-- user: string (nullable = true)
 |-- content: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- language: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- toot_length: integer (nullable = true)
 |-- hashtag: string (nullable = true)

+------------+----------+
|     hashtag|toot_count|
+------------+----------+
|          39|       313|
|  nowplaying|        25|
|issuecomment|        12|
|           m|         9|
|    Noticias|         5|
|     comment|         5|
|           1|         5|
|       Music|         3|
|Future_Diary|         3|
|     bangkok|         3|
+------------+----------+



# 4. Calculer les Agrégations (Nombre de Toots par Jour, Longueur Moyenne des Toots)

In [4]:
# Ajouter une colonne pour la longueur des toots
df_with_length = df_toots.withColumn("toot_length", F.length("content"))

# Compter le nombre de toots par jour
df_toots_per_day = df_toots.groupBy(F.to_date("timestamp").alias("date")).agg(  # Utilisez "timestamp" ici
    F.count("*").alias("total_toots")
)

# Calculer la longueur moyenne des toots
df_avg_toot_length = df_with_length.groupBy(F.to_date("timestamp").alias("date")).agg(  # Utilisez "timestamp" ici
    F.avg("toot_length").alias("avg_toot_length")
)

df_toots_per_day.show()
df_avg_toot_length.show()


+----------+-----------+
|      date|total_toots|
+----------+-----------+
|2024-10-02|          4|
|2022-03-15|          2|
|2024-09-30|          2|
|2024-10-08|      12312|
|2024-10-01|          1|
|2024-10-04|      20026|
|2024-10-07|       1763|
|2024-10-03|         45|
|2024-10-05|          1|
|2024-10-06|          2|
+----------+-----------+

+----------+-----------------+
|      date|  avg_toot_length|
+----------+-----------------+
|2024-10-02|           806.75|
|2022-03-15|           1913.5|
|2024-09-30|            809.5|
|2024-10-08|693.6509096816114|
|2024-10-01|            788.0|
|2024-10-04|692.6130030959753|
|2024-10-07| 573.354509359047|
|2024-10-03|870.3555555555556|
|2024-10-05|            912.0|
|2024-10-06|            802.0|
+----------+-----------------+

